<a href="https://colab.research.google.com/github/r7him/Abderrahmen0102/blob/main/Suppliers_warehouse01_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [808]:
!pip install gurobipy >10
from gurobipy import *
import numpy as np 
import pandas as np

In [809]:
# Warehouse demand in thousands of units
demand = [20, 10]
waste= [15, 30]
CL=[200, 30]

In [810]:
# supplier capacity in thousands of units
CS = [20, 15, 13,12]
#warehouse capacity
CW = [18, 22, 15]
#warehouse capacity
CG = [200, 100,50]
CO = [200, 100]

In [811]:
# Fixed costs for selectin a supplier.
Fs = [2000, 15000, 17000,10000]

In [812]:
# Fixed costs for each warehouses.
Fw = [16000, 15000,25000]

In [813]:
# Fixed costs for each storage.
Fg = [12000, 10000,17000]

In [814]:
# Fixed costs for each storage.
Fo = [12000, 2500]

In [815]:
# Fixed costs for each landfills.
Fl = [1200,25000]

In [816]:
# Transportation costs from suppliers to warehouses
TC_sw =       [[4000, 2000, 3000, 2500],
              [2500, 2600, 3400, 3000],
              [1200, 1800, 2600, 4100]]
# Transportation costs from warehouses to healthcare centres

TC_wh =      [[200, 150, 120],
              [250, 260,130]]

# Transportation costs of vaccine waste from to healthcare centres to waste storage.

TC_hg=       [[4000, 2000],
              [2500, 2600],
              [1200, 1800]]
# Transportation costs of vaccine waste from waste storage treatment storage.
TC_go=       [[200, 10000, 1500],
              [2500, 12000, 500]]

TC_ol=     [[40, 200000],
            [25000, 260000]]

In [817]:
# Range of plants and warehouses
suppliers = range(len(CS))
warehouses = range(len(CW))
healthcentre= range(len(demand))
healthwaste= range(len(waste))
storages    = range(len(CG))
treatments = range(len(waste))
landfills = range(len(CL))

In [818]:
m=Model()

In [819]:
#suppliers open decision variables: S_s[s] == 1 if supplier s is open.
S_s = m.addVars(suppliers,
                 vtype=GRB.BINARY,
                 obj=Fs,
                 name="S_s")
# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
# S_s  = []
# for s in suppliers:
#     S_s .append(m.addVar(vtype=GRB.BINARY,
#                          obj=fixedCost12[s],
#                          name="S_s[%d]" % s))

In [820]:
#Warehous open decision variables: W_w[w] == 1 if warehouse w is open.
W_w = m.addVars(warehouses,
                 vtype=GRB.BINARY,
                 obj=Fw,
                 name="W_w")

In [821]:
G_g = m.addVars(storages,
                vtype=GRB.BINARY, 
                obj=Fg , 
                name="G_g")

In [822]:
O_o = m.addVars(treatments,
                vtype=GRB.BINARY, 
                obj=Fo , 
                name="O_o")

In [823]:
L_l = m.addVars(landfills,
                vtype=GRB.BINARY, 
                obj=Fl , 
                name="L_l")

In [824]:
# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
(QS_sw)= m.addVars(warehouses,suppliers,obj=TC_sw, name="QS_sw")

In [825]:
# QW_w decision variables: QW_wh[w,p] captures the
# optimal quantity to transport to warehouse w from supplier s
QW_wh= m.addVars(healthcentre,warehouses, obj=TC_wh, name="QW_wh")


In [826]:
# QG_g decision variables: QW_wh[w,p] captures the
# optimal quantity to transport from healtcare to storages.
QH_hg= m.addVars(healthwaste,storages, obj=TC_hg, name="QH_hg")

In [827]:
QG_go= m.addVars(storages,treatments, obj=TC_go, name="QG_go")

In [828]:
QO_ol= m.addVars(landfills,treatments, obj=TC_ol, name="QL_ol")

In [829]:
# QW_w decision variables: QW_wh[w,p] captures the
# optimal quantity to transport to warehouse w from supplier s
#QU_wh= m.addVar(vtype=GRB.CONTINUOUS, name="QU_wh")

In [830]:
#m.setObjective((TC_sw[s,w] for s in TC_sw for w in TC_sw*QS_sw[w] for w in QS_sw), GRB.MINIMIZE)
m.ModelSense=GRB.MINIMIZE


In [831]:
# Production constraints # Note that the right-hand limit sets the production to zero if the plant # is closed
m.addConstrs(((QS_sw).sum('*', s) <= CS[s]*S_s[s] for s in suppliers), "Capacity of suppliers")
# Using Python looping constructs, the preceding would be...
# for s in suppliers:
#     m.addConstr(sum(QS_sw[w][s] for w in warehouses)
#                 <= capacity[s] * S_s[s], "Capacity[%d]" % s)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [832]:
# warehouses capacity constraints
m.addConstrs(((QW_wh).sum('*',w) <= CW[w]*W_w[w] for w in warehouses ), "Capacity of warehouses")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [833]:
# storges capacity constraints
m.addConstrs(((QH_hg).sum(g,'*') <= CG[g]*G_g[g] for g in storages ), "Capacity of storages")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [834]:
# treatment capacity constraints
m.addConstrs(((QG_go).sum('*',o) <= CO[o]*O_o[o] for o in treatments ), "Capacity of treatments")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [835]:
# landfill capacity constraints
m.addConstrs(((QO_ol).sum('*',l) <= CL[l]*L_l[l] for l in landfills ), "Capacity of landfills")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [836]:
# Demand constraints
m.addConstrs(
    (QW_wh.sum(h,'*') >= demand[h] for h in healthcentre), "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [837]:
# waste constraints
m.addConstrs(
   ( QH_hg.sum(g,'*') == waste[g] for g in healthwaste), "waste")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [838]:
#Flow Balance constraint between suppliers and warehouses
m.addConstrs((QS_sw.sum(w,'*') == QW_wh.sum('*',w) for w in warehouses), "flow balance")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [839]:
#Flow Balance constraint between storages and treatments
m.addConstrs((QG_go.sum(g,'*') == QH_hg.sum(g,'*') for g in storages), "flow balance")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [840]:
#Flow Balance constraint between treatments and landfills
m.addConstrs((QG_go.sum('*',l) == QO_ol.sum(l,'*') for l in landfills), "flow balance")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>}

In [841]:
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 26 rows, 48 columns and 100 nonzeros
Model fingerprint: 0xb7d2a5f3
Variable types: 34 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+01, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Found heuristic solution: objective 6369800.0000
Presolve removed 14 rows and 23 columns
Presolve time: 0.00s
Presolved: 12 rows, 25 columns, 49 nonzeros
Found heuristic solution: objective 320500.00000
Variable types: 18 continuous, 7 integer (7 binary)

Root relaxation: objective 2.554480e+05, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

In [842]:
for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

print('Obj: %g' % m.ObjVal)

S_s[0] 1
S_s[1] 1
S_s[2] -0
S_s[3] -0
W_w[0] 1
W_w[1] 0
W_w[2] 1
G_g[0] 1
G_g[1] 1
G_g[2] 0
O_o[0] 1
O_o[1] -0
L_l[0] 1
L_l[1] -0
QS_sw[0,0] 0
QS_sw[0,1] 15
QS_sw[0,2] 0
QS_sw[0,3] 0
QS_sw[1,0] 0
QS_sw[1,1] 0
QS_sw[1,2] 0
QS_sw[1,3] 0
QS_sw[2,0] 15
QS_sw[2,1] 0
QS_sw[2,2] 0
QS_sw[2,3] 0
QW_wh[0,0] 15
QW_wh[0,1] 0
QW_wh[0,2] 5
QW_wh[1,0] 0
QW_wh[1,1] 0
QW_wh[1,2] 10
QH_hg[0,0] 0
QH_hg[0,1] 15
QH_hg[0,2] 0
QH_hg[1,0] 0
QH_hg[1,1] 30
QH_hg[1,2] 0
QG_go[0,0] 15
QG_go[0,1] 0
QG_go[1,0] 30
QG_go[1,1] 0
QG_go[2,0] 0
QG_go[2,1] 0
QL_ol[0,0] 45
QL_ol[0,1] 0
QL_ol[1,0] 0
QL_ol[1,1] 0
Obj: 261900
